In [1]:
import pandas as pd
from pathlib import Path
from collections import Counter
from itertools import chain

In [37]:
base = Path('/media/ts/SSD_ubuntu/datasets/Clotho/labels')
splits = {'train': 'development', 'test': 'evaluation', 'val': 'validation'}

for split, filename in splits.items():
    df = pd.read_csv(base / f'clotho_captions_{filename}.csv')
    meta = pd.read_csv(base / f'clotho_metadata_{filename}.csv', engine='python', encoding='latin')

    df = df.merge(meta[['file_name', 'keywords']], on='file_name', how='inner')
    df['filename'] = df['file_name'].apply(lambda x: str(split / Path(x).with_suffix('.flac')))
    df['captions'] = df[[f'caption_{i}' for i in range(1, 6)]].values.tolist()
    df['tags'] = df['keywords'].str.split(';')
    df = df.drop(columns=[f'caption_{i}' for i in range(1, 6)] + ['file_name', 'keywords'])

    df.to_parquet((base.parent / ('annot_' + split)).with_suffix('.parquet'), index=False)

In [38]:
df = pd.read_parquet('/media/ts/SSD_ubuntu/datasets/Clotho/annot_train.parquet')
df

,filename,captions,tags
0,train/Distorted AM Radio noise.flac,"[A muddled noise of broken channel of the TV, ...","[noise, radio]"
1,train/Paper_Parchment_Rustling.flac,"[A person is turning a map over and over., A p...","[movement, parchment, scroll, crinkle, pages, ..."
2,train/03 Whales Slowing Down.flac,[Several barnyard animals mooing in a barn whi...,"[boat, voice, earth, sea, humpback-whales, liv..."
3,train/Rope tied to boat in port.flac,[An office chair is squeaking as someone bends...,"[tanker, sail-boat, rope, ocean, shipping, yac..."
4,train/carpenter bee.flac,[A flying bee is buzzing loudly around an obje...,"[bugs, field-recording, insects]"
...,...,...,...
3834,train/Metallic Ping CPU Heatsink.flac,[Tools are being tried to make different sound...,"[cpu, heatsink, metallic, ping]"
3835,train/Fumbling.flac,"[multiple items are picked up, and a tin pan i...","[junk, crash, items, metal, searching, plastic..."
3836,train/cold waterdrops in a hot pot.flac,[Someone is flipping over food on a hot grill....,"[drop, hiss, hot-plate, pot, steam, water]"
3837,train/Dry leaves falling on cement floor.flac,[Tapping noises are being made before paper is...,"[crinkle, falling, OWI, fall, dropping, crisp,..."


In [34]:
Counter(chain.from_iterable(df.tags.to_list())).most_common(10)

[('field-recording', 313),
 ('water', 138),
 ('nature', 98),
 ('ambience', 84),
 ('ambient', 81),
 ('birds', 80),
 ('noise', 76),
 ('city', 66),
 ('people', 55),
 ('wind', 55)]